## Hand-coded Neural network (better than baseline but still bad)

We will train the neural network to identify the pictures by implementing backprogration and gradient descent + tuning (try different number of hidden layers, number of nodes,..)

![title](img/nn.jpg)

We got 56% accuracy on the real test set provided by Kaggle, which is better than SVM but it's still not good enough. Reason: NN does not take neighborhood into consideration. It loses a lot of information when flatting out the pictures.

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import os
!unzip data.zip

Archive:  data.zip
  inflating: xTest.npy               
  inflating: xTrain.npy              
  inflating: yTrain.npy              


In [38]:
#Load images with numpy
yTrain = np.array(np.load("../yTrain.npy"))
xTrain = np.load("../xTrain.npy")
xTest = np.load("../xTest.npy")
print(yTrain.shape)
print(xTrain.shape)
print(xTest.shape)

(10000, 1)
(10000, 1444)
(10000, 1444)


In [39]:
# make dict {0:"sink",1:"pear",...} 
categories=["sink","pear","moustache","nose","skateboard","penguin","peanut","skull","panda","paintbrush","nail","apple","rifle","mug","sailboat","pineapple","spoon","rabbit","shovel","rollerskates","screwdriver","scorpion","rhinoceros","pool","octagon","pillow","parrot","squiggle","mouth","empty","pencil"]
categories_dict = dict(enumerate(categories))
print(categories_dict)

{0: 'sink', 1: 'pear', 2: 'moustache', 3: 'nose', 4: 'skateboard', 5: 'penguin', 6: 'peanut', 7: 'skull', 8: 'panda', 9: 'paintbrush', 10: 'nail', 11: 'apple', 12: 'rifle', 13: 'mug', 14: 'sailboat', 15: 'pineapple', 16: 'spoon', 17: 'rabbit', 18: 'shovel', 19: 'rollerskates', 20: 'screwdriver', 21: 'scorpion', 22: 'rhinoceros', 23: 'pool', 24: 'octagon', 25: 'pillow', 26: 'parrot', 27: 'squiggle', 28: 'mouth', 29: 'empty', 30: 'pencil'}


In [61]:
#Single Layer NN
import csv
import numpy as np
from sklearn.metrics import accuracy_score
import random

from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

#Defining the functions for sigmoid and its dervitive
#The derivitive assumes it is being fed the output of a sigmoid function
def sigmoid(x):
    return 1.0/(1 + np.exp(-x))
def der_sigmoid(x):
    return (x * (1-x))

#                   The Neural Network Class
class NeuralNetwork:
      #Initialize the variable
    def __init__(self,x,y,step,nodes):
        self.xTrain = x
        self.yTrain = y
        self.num_nodes = nodes
        self.learning_rate = step
        self.w1 = np.random.uniform(-1,1,(len(self.xTrain[0]),self.num_nodes))
        self.w2 = np.random.uniform(-1,1,(self.num_nodes,31))
        self.output = np.zeros(31)
        self.der_w1 = np.zeros((len(self.xTrain[0]),self.num_nodes))
        self.der_w2 = np.zeros((self.num_nodes,31))
        self.current_point = 0 #This value represents the index of the current sample being inputed
    
      #Computes the output for each sample
    def feedforward(self,cur):
        self.current_point = cur
        self.hidden_layer = sigmoid(np.dot(self.xTrain[self.current_point], self.w1))
        self.output = sigmoid(np.dot(self.hidden_layer,self.w2))
  
  #Computes the derivitive for each weight at each sample
    def backprop(self):
        ders_output = np.zeros((31,31))
        ders_hidden = np.zeros((self.num_nodes,self.num_nodes))
        errors = np.zeros(31)
    
    #Set the diagonal matrices with the left side of the nodes of each layer
    for i in range(31):
        ders_output[i][i] = der_sigmoid(self.output[i])
        errors[i] = (self.output[i] - self.yTrain[self.current_point][i])
    for i in range(self.num_nodes):
        ders_hidden[i][i] = der_sigmoid(self.hidden_layer[i])
    
    #Compute the backprop derivitive values
    output_backprop = np.matmul(ders_output,errors)
    hidden_backprop = np.matmul(ders_hidden,np.matmul(self.w2,output_backprop))
    
    #Compute the change to the weights and then update the weights
    change_in_w2 = -1.0 * self.learning_rate * np.matmul(self.hidden_layer.reshape(self.num_nodes,1),output_backprop.reshape(1,31))
    change_in_w1 = -1.0 * self.learning_rate * np.matmul(self.xTrain[self.current_point].reshape(1444,1),hidden_backprop.reshape(1,self.num_nodes))
    
    self.w1 = self.w1 + change_in_w1
    self.w2 = self.w2 + change_in_w2
                           
#A function that calculates the accuracy with given weights and data. Used to get accuracy on validation set   
def valid_calc(weight1,weight2,xValid,yValid):
    hidden = sigmoid(np.dot(xValid,weight1))
    result = sigmoid(np.dot(hidden,weight2))
    guesses = []
    for i in result:
        guesses.append(np.argmax(i))
        guess = np.array(guesses)
        acc = accuracy_score(yValid,guess)
        return acc,guess

#A function used to calculate the predictions on the test set
def test_calc(weight1,weight2,x_Test):
    hidden = sigmoid(np.dot(x_Test,weight1))
    result = sigmoid(np.dot(hidden,weight2))
    guesses = []
    for i in result:
    guesses.append(np.argmax(i))
    guess = np.array(guesses)
    return guess

#Run the network. Currently runs all hyper parameters combinations.
# This can be extremely time consuming on a regular GPU, so running a few one at a time
# may be an easier option
with open("../layer_1.csv","w") as f:
    writer = csv.writer(f)
    current_loss = 0
    num_node_options = [50,100,200,500]
    learning_rate_options = [.001,.01,.1,1]
    for nodes in num_node_options:
        for rate in learning_rate_options:
            print("======Running Neural Network with", nodes, " HIDDEN NODES and a " , rate, " LEARNING RATE")
            average_accuracy = 0
            index = 0
          #Splits the training set into 5-folds and runs the Network 5 teams, each time with a different valdiation set
            for train_index, valid_index in kf.split(xTrain):
                index+=1
                x_train, x_valid = xTrain[train_index], xTrain[valid_index]
                y_train, y_valid = yTrain[train_index], yTrain[valid_index]
                y_train = y_train.ravel()
                y_valid = y_valid.ravel()

            #Converts the y_train into a 2-d array as mentioned in the report
            newY = np.zeros((8000,31))
            for i in range(len(y_train)):
                newY[i][y_train[i]] = 1

            print("On CV ", index)
            nn = NeuralNetwork(x_train,newY,rate,nodes)
            for i in range(200):
                accuracy_to_write, other = valid_calc(nn.w1,nn.w2,x_valid,y_valid)
                writer.writerow([accuracy_to_write])
                if(i%10==0):
                    print("Currently on epoch: ", i)
                    print(valid_calc(nn.w1,nn.w2,x_valid,y_valid))
                #Shuffles the order the network visits each sample for proper stochastic gradient descent
                rangeNum = np.linspace(0,7999,8000)
                random.shuffle(rangeNum)
                for j in rangeNum:
                    j = int(j)
                    nn.feedforward(j)
                    nn.backprop()
            #Computes the accuracy and averages it at the end
            accuracy,gues = valid_calc(nn.w1,nn.w2,x_valid,y_valid)
            average_accuracy += accuracy
            average_accuracy = average_accuracy/5
            writer.writerow(["Score for", nodes, "nodes with a ",rate,"learning rate", average_accuracy])
    
#This gives us our predictions which can be written into a csv file to be submitted
yPredictions = test_calc(nn.w1,nn.w2,xTest)

======Running Neural Network with 50  HIDDEN NODES and a  0.001  LEARNING RATE
On CV  1
Currently on epoch:  0
(0.0195, array([14, 17,  3, ...,  1, 30, 21]))
Currently on epoch:  10
(0.0325, array([14, 28, 23, ..., 12, 22, 23]))
Currently on epoch:  20
(0.0425, array([14, 28, 23, ..., 24, 28, 23]))


KeyboardInterrupt: ignored

In [0]:
#Double Layer NN
#The code for the neural networks of different layers is identical to that of
# the single layered neural network except that more layers and corresponding weights
# are added, and the. New valid_calc functions also had to be created to accomodiate the
# additional weights and layers.
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import random

class NeuralNetwork_2:
    def __init__(self,x,y,step,nodes,nodes_2):
        self.xTrain = x
        self.yTrain = y
        self.num_nodes = nodes
        self.num_nodes_2 = nodes_2
        self.learning_rate = step
        self.w1 = np.random.uniform(-1,1,(len(self.xTrain[0]),self.num_nodes))
        self.w2 = np.random.uniform(-1,1,(self.num_nodes,self.num_nodes_2))
        self.w3 = np.random.uniform(-1,1,(self.num_nodes_2,31))
        self.output = np.zeros(31)
        self.der_w1 = np.zeros((len(self.xTrain[0]),self.num_nodes))
        self.der_w2 = np.zeros((self.num_nodes,self.num_nodes_2))
        self.der_w3 = np.zeros((self.num_nodes_2,31))
        self.current_point = 0
    
    def feedforward(self,cur):
        self.current_point = cur
        self.hidden_layer = sigmoid(np.dot(self.xTrain[self.current_point], self.w1))
        self.hidden_layer_2 = sigmoid(np.dot(self.hidden_layer,self.w2))
        self.output = sigmoid(np.dot(self.hidden_layer_2,self.w3))
    
    def backprop(self):
        ders_output = np.zeros((31,31))
        ders_hidden_2 = np.zeros((self.num_nodes_2,self.num_nodes_2))
        ders_hidden = np.zeros((self.num_nodes,self.num_nodes))
        errors = np.zeros(31)
    
    for i in range(31):
        ders_output[i][i] = der_sigmoid(self.output[i])
        errors[i] = (self.output[i] - self.yTrain[self.current_point][i])
    for i in range(self.num_nodes_2):
        ders_hidden_2[i][i] = der_sigmoid(self.hidden_layer_2[i])
    for i in range(self.num_nodes):
        ders_hidden[i][i] = der_sigmoid(self.hidden_layer[i])
      
    output_backprop = np.matmul(ders_output,errors)
    hidden_2_backprop = np.matmul(ders_hidden_2,np.matmul(self.w3,output_backprop))
    hidden_backprop = np.matmul(ders_hidden,np.matmul(self.w2,hidden_2_backprop))
    
    change_in_w3 = -1.0 * self.learning_rate * np.matmul(self.hidden_layer_2.reshape(self.num_nodes_2,1),output_backprop.reshape(1,31))
    change_in_w2 = -1.0 * self.learning_rate * np.matmul(self.hidden_layer.reshape(self.num_nodes,1),hidden_2_backprop.reshape(1,self.num_nodes_2))
    change_in_w1 = -1.0 * self.learning_rate * np.matmul(self.xTrain[self.current_point].reshape(1444,1),hidden_backprop.reshape(1,self.num_nodes))
    
    self.w1 = self.w1 + change_in_w1
    self.w2 = self.w2 + change_in_w2
    self.w3 = self.w3 + change_in_w3

def valid_calc_2(weight1,weight2,weight3,xValid,yValid):
    hidden = sigmoid(np.dot(xValid,weight1))
    hidden_2 = sigmoid(np.dot(hidden,weight2))
    result = sigmoid(np.dot(hidden_2,weight3))
    guesses = []
    for i in result:
        guesses.append(np.argmax(i))
    guess = np.array(guesses)
    acc = accuracy_score(yValid,guess)
    return acc,guess

In [48]:
#Run the network for 2 layers
kf = KFold(n_splits=5)
with open("../content/layer_2.csv","w") as f:
    writer = csv.writer(f)
    current_loss = 0
  
  #Currently run with just our best number of nodes and learning rate, but more can be easily added
    num_node_options = [100]
    learning_rate_options = [.1]
    for nodes in num_node_options:
        for rate in learning_rate_options:
            print("======Running 2 Layer Neural Network with", nodes, " HIDDEN NODES and a ",rate," LEARNING RATE")
            average_accuracy = 0
            index = 0
            for train_index, valid_index in kf.split(xTrain):
                index+=1
                x_train, x_valid = xTrain[train_index], xTrain[valid_index]
                y_train, y_valid = yTrain[train_index], yTrain[valid_index]
                y_train = y_train.ravel()
                y_valid = y_valid.ravel()
                newY = np.zeros((8000,31))
                for i in range(len(y_train)):
                    newY[i][y_train[i]] = 1
                print("ON CV ", index)
                nn = NeuralNetwork_2(x_train,newY,rate,nodes,nodes)
                for i in range(100):
                    acc_for_writing,other = valid_calc_2(nn.w1,nn.w2,nn.w3,x_valid,y_valid)
                    writer.writerow([acc_for_writing])
                    if(i%10==0):
                        print("Currently on epoch: ", i)
                        print(valid_calc_2(nn.w1,nn.w2,nn.w3,x_valid,y_valid))
                    rangeNum = np.linspace(0,7999,8000)
                    random.shuffle(rangeNum)
                    for j in rangeNum:
                        j = int(j)
                        nn.feedforward(j)
                        nn.backprop()
                accuracy,gues = valid_calc_2(nn.w1,nn.w2,nn.w3,x_valid,y_valid)
                average_accuracy += accuracy
            average_accuracy = average_accuracy/5

======Running 2 Layer Neural Network with 100  HIDDEN NODES and a  0.1  LEARNING RATE
ON CV  1
Currently on epoch:  0
(0.024, array([ 6,  6, 20, ...,  5, 20, 19]))
Currently on epoch:  10
(0.401, array([24, 14, 21, ..., 15, 30, 13]))
Currently on epoch:  20
(0.417, array([ 1, 14, 21, ..., 15, 20, 13]))
Currently on epoch:  30
(0.4245, array([ 1, 27, 21, ..., 15, 30, 13]))


KeyboardInterrupt: ignored

In [0]:
#Triple Layer NN
import numpy as np
from sklearn.metrics import accuracy_score

class NeuralNetwork_3:
    def __init__(self,x,y,step,nodes,nodes_2,nodes_3):
        self.xTrain = x
        self.yTrain = y
        self.num_nodes = nodes
        self.num_nodes_2 = nodes_2
        self.num_nodes_3 = nodes_3
        self.learning_rate = step
        self.w1 = np.random.uniform(-1,1,(len(self.xTrain[0]),self.num_nodes))
        self.w2 = np.random.uniform(-1,1,(self.num_nodes,self.num_nodes_2))
        self.w3 = np.random.uniform(-1,1,(self.num_nodes_2,self.num_nodes_3))
        self.w4 = np.random.uniform(-1,1,(self.num_nodes_3,31))
        self.output = np.zeros(31)
        self.der_w1 = np.zeros((len(self.xTrain[0]),self.num_nodes))
        self.der_w2 = np.zeros((self.num_nodes,self.num_nodes_2))
        self.der_w3 = np.zeros((self.num_nodes_2,self.num_nodes_3))
        self.der_w4 = np.zeros((self.num_nodes_3,31))
        self.current_point = 0
    
    def feedforward(self,cur):
        self.current_point = cur
        self.hidden_layer = sigmoid(np.dot(self.xTrain[self.current_point], self.w1))
        self.hidden_layer_2 = sigmoid(np.dot(self.hidden_layer,self.w2))
        self.hidden_layer_3 = sigmoid(np.dot(self.hidden_layer_2,self.w3))
        self.output = sigmoid(np.dot(self.hidden_layer_3,self.w4))
    
    def backprop(self):
        ders_output = np.zeros((31,31))
        ders_hidden_3 = np.zeros((self.num_nodes_3,self.num_nodes_3))
        ders_hidden_2 = np.zeros((self.num_nodes_2,self.num_nodes_2))
        ders_hidden = np.zeros((self.num_nodes,self.num_nodes))
        errors = np.zeros(31)
    
    for i in range(31):
        ders_output[i][i] = der_sigmoid(self.output[i])
        errors[i] = (self.output[i] - self.yTrain[self.current_point][i])
    for i in range(self.num_nodes_3):
        ders_hidden_3[i][i] = der_sigmoid(self.hidden_layer_3[i])
    for i in range(self.num_nodes_2):
        ders_hidden_2[i][i] = der_sigmoid(self.hidden_layer_2[i])
    for i in range(self.num_nodes):
        ders_hidden[i][i] = der_sigmoid(self.hidden_layer[i])
      
    output_backprop = np.matmul(ders_output,errors)
    hidden_3_backprop = np.matmul(ders_hidden_3,np.matmul(self.w4,output_backprop))
    hidden_2_backprop = np.matmul(ders_hidden_2,np.matmul(self.w3,hidden_3_backprop))
    hidden_backprop = np.matmul(ders_hidden,np.matmul(self.w2,hidden_2_backprop))
    
    change_in_w4 = -1.0 * self.learning_rate * np.matmul(self.hidden_layer_3.reshape(self.num_nodes_3,1),output_backprop.reshape(1,31))
    change_in_w3 = -1.0 * self.learning_rate * np.matmul(self.hidden_layer_2.reshape(self.num_nodes_2,1),hidden_3_backprop.reshape(1,self.num_nodes_3))
    change_in_w2 = -1.0 * self.learning_rate * np.matmul(self.hidden_layer.reshape(self.num_nodes,1),hidden_2_backprop.reshape(1,self.num_nodes_2))
    change_in_w1 = -1.0 * self.learning_rate * np.matmul(self.xTrain[self.current_point].reshape(1444,1),hidden_backprop.reshape(1,self.num_nodes))
    
    self.w1 = self.w1 + change_in_w1
    self.w2 = self.w2 + change_in_w2
    self.w3 = self.w3 + change_in_w3
    self.w4 = self.w4 + change_in_w4

    
def valid_calc_3(weight1,weight2,weight3,weight4,xValid,yValid):
    hidden = sigmoid(np.dot(xValid,weight1))
    hidden_2 = sigmoid(np.dot(hidden,weight2))
    hidden_3 = sigmoid(np.dot(hidden_2,weight3))
    result = sigmoid(np.dot(hidden_3,weight4))
    guesses = []
    for i in result:
        guesses.append(np.argmax(i))
    guess = np.array(guesses)
    acc = accuracy_score(yValid,guess)
    return acc,guess



In [59]:
#Run the network for 3 layers
kf = KFold(n_splits=5)
with open("../content/layer_3.csv","w") as f:
  writer = csv.writer(f)
  current_loss = 0
  num_node_options = [100]
  learning_rate_options = [.1]
  for nodes in num_node_options:
    for rate in learning_rate_options:
      print("======Running a 3 layer Neural Network with", nodes, " HIDDEN NODES and a ",rate," LEARNING RATE")
      average_accuracy = 0
      index = 0
      for train_index, valid_index in kf.split(xTrain):
        index+=1
        x_train, x_valid = xTrain[train_index], xTrain[valid_index]
        y_train, y_valid = yTrain[train_index], yTrain[valid_index]
        y_train = y_train.ravel()
        y_valid = y_valid.ravel()
        newY = np.zeros((8000,31))
        for i in range(len(y_train)):
          newY[i][y_train[i]] = 1
        print("ON CV ", index)
        nn = NeuralNetwork_3(x_train,newY,rate,nodes,nodes,nodes)
        for i in range(100):
          if(i%10==0):
            print("Currently on epoch: ", i)
            print(valid_calc_3(nn.w1,nn.w2,nn.w3,nn.w4,x_valid,y_valid))
          rangeNum = np.linspace(0,7999,8000)
          random.shuffle(rangeNum)
          for j in rangeNum:
            j = int(j)
            nn.feedforward(j)
            nn.backprop()
        accuracy,gues = valid_calc_3(nn.w1,nn.w2,nn.w3,nn.w4,x_valid,y_valid)
        average_accuracy += accuracy
      average_accuracy = average_accuracy/5
      writer.writerow(["Score for", nodes, "nodes with a", rate, "learning rate", average_accuracy])

======Running a 3 layer Neural Network with 100  HIDDEN NODES and a  0.1  LEARNING RATE
ON CV  1
Currently on epoch:  0
(0.0345, array([10, 10, 10, ..., 26, 26, 10]))


KeyboardInterrupt: ignored

In [0]:
#Quadruple Layer NN
import numpy as np
from sklearn.metrics import accuracy_score

class NeuralNetwork_4:
  def __init__(self,x,y,step,nodes,nodes_2,nodes_3,nodes_4):
    self.xTrain = x
    self.yTrain = y
    self.num_nodes = nodes
    self.num_nodes_2 = nodes_2
    self.num_nodes_3 = nodes_3
    self.num_nodes_4 = nodes_4
    self.learning_rate = step
    self.w1 = np.random.uniform(-1,1,(len(self.xTrain[0]),self.num_nodes))
    self.w2 = np.random.uniform(-1,1,(self.num_nodes,self.num_nodes_2))
    self.w3 = np.random.uniform(-1,1,(self.num_nodes_2,self.num_nodes_3))
    self.w4 = np.random.uniform(-1,1,(self.num_nodes_3,self.num_nodes_4))
    self.w5 = np.random.uniform(-1,1,(self.num_nodes_4,31))
    
    self.output = np.zeros(31)
    self.der_w1 = np.zeros((len(self.xTrain[0]),self.num_nodes))
    self.der_w2 = np.zeros((self.num_nodes,self.num_nodes_2))
    self.der_w3 = np.zeros((self.num_nodes_2,self.num_nodes_3))
    self.der_w4 = np.zeros((self.num_nodes_3,self.num_nodes_4))
    self.der_w5 = np.zeros((self.num_nodes_4,31))
    
    self.current_point = 0
    
  def feedforward(self,cur):
    self.current_point = cur
    self.hidden_layer = sigmoid(np.dot(self.xTrain[self.current_point], self.w1))
    self.hidden_layer_2 = sigmoid(np.dot(self.hidden_layer,self.w2))
    self.hidden_layer_3 = sigmoid(np.dot(self.hidden_layer_2,self.w3))
    self.hidden_layer_4 = sigmoid(np.dot(self.hidden_layer_3,self.w4))
    self.output = sigmoid(np.dot(self.hidden_layer_4,self.w5))
    
  def backprop(self):
    ders_output = np.zeros((31,31))
    ders_hidden_4 = np.zeros((self.num_nodes_4,self.num_nodes_4))
    ders_hidden_3 = np.zeros((self.num_nodes_3,self.num_nodes_3))
    ders_hidden_2 = np.zeros((self.num_nodes_2,self.num_nodes_2))
    ders_hidden = np.zeros((self.num_nodes,self.num_nodes))
    errors = np.zeros(31)
    
    for i in range(31):
      ders_output[i][i] = der_sigmoid(self.output[i])
      errors[i] = (self.output[i] - self.yTrain[self.current_point][i])
    for i in range(self.num_nodes_4):
      ders_hidden_4[i][i] = der_sigmoid(self.hidden_layer_4[i])
    for i in range(self.num_nodes_3):
      ders_hidden_3[i][i] = der_sigmoid(self.hidden_layer_3[i])
    for i in range(self.num_nodes_2):
      ders_hidden_2[i][i] = der_sigmoid(self.hidden_layer_2[i])
    for i in range(self.num_nodes):
      ders_hidden[i][i] = der_sigmoid(self.hidden_layer[i])
      
    output_backprop = np.matmul(ders_output,errors)
    hidden_4_backprop = np.matmul(ders_hidden_4,np.matmul(self.w5,output_backprop))
    hidden_3_backprop = np.matmul(ders_hidden_3,np.matmul(self.w4,hidden_4_backprop))
    hidden_2_backprop = np.matmul(ders_hidden_2,np.matmul(self.w3,hidden_3_backprop))
    hidden_backprop = np.matmul(ders_hidden,np.matmul(self.w2,hidden_2_backprop))
    
    change_in_w5 = -1.0 * self.learning_rate * np.matmul(self.hidden_layer_4.reshape(self.num_nodes_4,1),output_backprop.reshape(1,31))
    change_in_w4 = -1.0 * self.learning_rate * np.matmul(self.hidden_layer_3.reshape(self.num_nodes_3,1),hidden_4_backprop.reshape(1,self.num_nodes_4))
    change_in_w3 = -1.0 * self.learning_rate * np.matmul(self.hidden_layer_2.reshape(self.num_nodes_2,1),hidden_3_backprop.reshape(1,self.num_nodes_3))
    change_in_w2 = -1.0 * self.learning_rate * np.matmul(self.hidden_layer.reshape(self.num_nodes,1),hidden_2_backprop.reshape(1,self.num_nodes_2))
    change_in_w1 = -1.0 * self.learning_rate * np.matmul(self.xTrain[self.current_point].reshape(1444,1),hidden_backprop.reshape(1,self.num_nodes))
    
    self.w1 = self.w1 + change_in_w1
    self.w2 = self.w2 + change_in_w2
    self.w3 = self.w3 + change_in_w3
    self.w4 = self.w4 + change_in_w4
    self.w5 = self.w5 + change_in_w5
                           
    
def valid_calc_4(weight1,weight2,weight3,weight4,weight5,xValid,yValid):
  hidden = sigmoid(np.dot(xValid,weight1))
  hidden_2 = sigmoid(np.dot(hidden,weight2))
  hidden_3 = sigmoid(np.dot(hidden_2,weight3))
  hidden_4 = sigmoid(np.dot(hidden_3,weight4))
  result = sigmoid(np.dot(hidden_4,weight5))
  guesses = []
  for i in result:
    guesses.append(np.argmax(i))
  guess = np.array(guesses)
  acc = accuracy_score(yValid,guess)
  return acc,guess

In [54]:
#Run the network for 4 layers
kf = KFold(n_splits=5)
with open("../layer_4.csv","w") as f:
  writer = csv.writer(f)
  current_loss = 0
  num_node_options = [100]
  learning_rate_options = [.1]
  for nodes in num_node_options:
    for rate in learning_rate_options:
      print("======Running a 4 layer Neural Network with ",nodes," HIDDEN NODES and a ",rate," LEARNING RATE")
      average_accuracy = 0
      index = 0
      for train_index, valid_index in kf.split(xTrain):
        index+=1
        x_train, x_valid = xTrain[train_index], xTrain[valid_index]
        y_train, y_valid = yTrain[train_index], yTrain[valid_index]
        y_train = y_train.ravel()
        y_valid = y_valid.ravel()
        newY = np.zeros((8000,31))
        for i in range(len(y_train)):
          newY[i][y_train[i]] = 1
        print("On CV ", index)
        nn = NeuralNetwork_4(x_train,newY,rate,nodes,nodes,nodes,nodes)
        for i in range(100):
          if(i%10==0):
            print("Currently on epoch: ", i)
            print(valid_calc_4(nn.w1,nn.w2,nn.w3,nn.w4,nn.w5,x_valid,y_valid))
          rangeNum = np.linspace(0,7999,8000)
          random.shuffle(rangeNum)
          for j in rangeNum:
            j = int(j)
            nn.feedforward(j)
            nn.backprop()
        accuracy,gues = valid_calc_4(nn.w1,nn.w2,nn.w3,nn.w4,nn.w5x_valid,y_valid)
        average_accuracy += accuracy
      average_accuracy = average_accuracy/5
      writer.writerow(["Score for ",nodes," nodes with a ",rate," learning rate", average_accuracy])

======Running a 4 layer Neural Network with  100  HIDDEN NODES and a  0.1  LEARNING RATE
On CV  1
Currently on epoch:  0
(0.04, array([17, 22, 22, ..., 17, 17, 17]))
Currently on epoch:  10
(0.379, array([18,  2, 21, ...,  5, 17, 13]))


KeyboardInterrupt: ignored

In [0]:
#Quintuple Layer NN
import numpy as np
from sklearn.metrics import accuracy_score

class NeuralNetwork_5:
  def __init__(self,x,y,step,nodes,nodes_2,nodes_3,nodes_4,nodes_5):
    self.xTrain = x
    self.yTrain = y
    self.num_nodes = nodes
    self.num_nodes_2 = nodes_2
    self.num_nodes_3 = nodes_3
    self.num_nodes_4 = nodes_4
    self.num_nodes_5 = nodes_5
    self.learning_rate = step
    self.w1 = np.random.uniform(-1,1,(len(self.xTrain[0]),self.num_nodes))
    self.w2 = np.random.uniform(-1,1,(self.num_nodes,self.num_nodes_2))
    self.w3 = np.random.uniform(-1,1,(self.num_nodes_2,self.num_nodes_3))
    self.w4 = np.random.uniform(-1,1,(self.num_nodes_3,self.num_nodes_4))
    self.w5 = np.random.uniform(-1,1,(self.num_nodes_4,self.num_nodes_5))
    self.w6 = np.random.uniform(-1,1,(self.num_nodes_5,31))
    
    self.output = np.zeros(31)
    self.der_w1 = np.zeros((len(self.xTrain[0]),self.num_nodes))
    self.der_w2 = np.zeros((self.num_nodes,self.num_nodes_2))
    self.der_w3 = np.zeros((self.num_nodes_2,self.num_nodes_3))
    self.der_w4 = np.zeros((self.num_nodes_3,self.num_nodes_4))
    self.der_w5 = np.zeros((self.num_nodes_4,self.num_nodes_5))
    self.der_w6 = np.zeros((self.num_nodes_5,31))
    
    self.current_point = 0
    
  def feedforward(self,cur):
    self.current_point = cur
    self.hidden_layer = sigmoid(np.dot(self.xTrain[self.current_point], self.w1))
    self.hidden_layer_2 = sigmoid(np.dot(self.hidden_layer,self.w2))
    self.hidden_layer_3 = sigmoid(np.dot(self.hidden_layer_2,self.w3))
    self.hidden_layer_4 = sigmoid(np.dot(self.hidden_layer_3,self.w4))
    self.hidden_layer_5 = sigmoid(np.dot(self.hidden_layer_4,self.w5))
    self.output = sigmoid(np.dot(self.hidden_layer_5,self.w6))
    
  def backprop(self):
    ders_output = np.zeros((31,31))
    ders_hidden_5 = np.zeros((self.num_nodes_5,self.num_nodes_5))
    ders_hidden_4 = np.zeros((self.num_nodes_4,self.num_nodes_4))
    ders_hidden_3 = np.zeros((self.num_nodes_3,self.num_nodes_3))
    ders_hidden_2 = np.zeros((self.num_nodes_2,self.num_nodes_2))
    ders_hidden = np.zeros((self.num_nodes,self.num_nodes))
    errors = np.zeros(31)
    
    for i in range(31):
      ders_output[i][i] = der_sigmoid(self.output[i])
      errors[i] = (self.output[i] - self.yTrain[self.current_point][i])
    for i in range(self.num_nodes_5):
      ders_hidden_5[i][i] = der_sigmoid(self.hidden_layer_5[i])
    for i in range(self.num_nodes_4):
      ders_hidden_4[i][i] = der_sigmoid(self.hidden_layer_4[i])
    for i in range(self.num_nodes_3):
      ders_hidden_3[i][i] = der_sigmoid(self.hidden_layer_3[i])
    for i in range(self.num_nodes_2):
      ders_hidden_2[i][i] = der_sigmoid(self.hidden_layer_2[i])
    for i in range(self.num_nodes):
      ders_hidden[i][i] = der_sigmoid(self.hidden_layer[i])
      
    output_backprop = np.matmul(ders_output,errors)
    hidden_5_backprop = np.matmul(ders_hidden_5,np.matmul(self.w6,output_backprop))
    hidden_4_backprop = np.matmul(ders_hidden_4,np.matmul(self.w5,hidden_5_backprop))
    hidden_3_backprop = np.matmul(ders_hidden_3,np.matmul(self.w4,hidden_4_backprop))
    hidden_2_backprop = np.matmul(ders_hidden_2,np.matmul(self.w3,hidden_3_backprop))
    hidden_backprop = np.matmul(ders_hidden,np.matmul(self.w2,hidden_2_backprop))
    
    change_in_w6 = -1.0 * self.learning_rate * np.matmul(self.hidden_layer_5.reshape(self.num_nodes_4,1),output_backprop.reshape(1,31))
    change_in_w5 = -1.0 * self.learning_rate * np.matmul(self.hidden_layer_4.reshape(self.num_nodes_4,1),hidden_5_backprop.reshape(1,self.num_nodes_5))
    change_in_w4 = -1.0 * self.learning_rate * np.matmul(self.hidden_layer_3.reshape(self.num_nodes_3,1),hidden_4_backprop.reshape(1,self.num_nodes_4))
    change_in_w3 = -1.0 * self.learning_rate * np.matmul(self.hidden_layer_2.reshape(self.num_nodes_2,1),hidden_3_backprop.reshape(1,self.num_nodes_3))
    change_in_w2 = -1.0 * self.learning_rate * np.matmul(self.hidden_layer.reshape(self.num_nodes,1),hidden_2_backprop.reshape(1,self.num_nodes_2))
    change_in_w1 = -1.0 * self.learning_rate * np.matmul(self.xTrain[self.current_point].reshape(1444,1),hidden_backprop.reshape(1,self.num_nodes))
    
    self.w1 = self.w1 + change_in_w1
    self.w2 = self.w2 + change_in_w2
    self.w3 = self.w3 + change_in_w3
    self.w4 = self.w4 + change_in_w4
    self.w5 = self.w5 + change_in_w5
    self.w6 = self.w6 + change_in_w6
    
def valid_calc_5(weight1,weight2,weight3,weight4,weight5,weight6,xValid,yValid):
  hidden = sigmoid(np.dot(xValid,weight1))
  hidden_2 = sigmoid(np.dot(hidden,weight2))
  hidden_3 = sigmoid(np.dot(hidden_2,weight3))
  hidden_4 = sigmoid(np.dot(hidden_3,weight4))
  hidden_5 = sigmoid(np.dot(hidden_4,weight5))
  result = sigmoid(np.dot(hidden_5,weight6))
  guesses = []
  for i in result:
    guesses.append(np.argmax(i))
  guess = np.array(guesses)
  acc = accuracy_score(yValid,guess)
  return acc,guess


In [56]:
#Run the network for 5 layers
with open("../layer_5.csv","w") as f:
  writer = csv.writer(f)
  current_loss = 0
  num_node_options = [100]
  learning_rate_options = [.1]
  for nodes in num_node_options:
    for rate in learning_rate_options:
      print("======Running a 5 layer Neural Network with", nodes, " HIDDEN NODES and a ",rate," LEARNING RATE")
      average_accuracy = 0
      index = 0
      for train_index, valid_index in kf.split(xTrain):
        index+=1
        x_train, x_valid = xTrain[train_index], xTrain[valid_index]
        y_train, y_valid = yTrain[train_index], yTrain[valid_index]
        y_train = y_train.ravel()
        y_valid = y_valid.ravel()
        newY = np.zeros((8000,31))
        for i in range(len(y_train)):
          newY[i][y_train[i]] = 1
        print("ON CV ", index)
        nn = NeuralNetwork_5(x_train,newY,rate,nodes,nodes,nodes,nodes,nodes)
        for i in range(100):
          if(i%10==0):
            print("Currently on epoch: ", i)
            print(valid_calc_5(nn.w1,nn.w2,nn.w3,nn.w4,nn.w5,nn.w6,x_valid,y_valid))
          rangeNum = np.linspace(0,7999,8000)
          random.shuffle(rangeNum)
          for j in rangeNum:
            j = int(j)
            nn.feedforward(j)
            nn.backprop()
        accuracy,gues = valid_calc_5(nn.w1,nn.w2,nn.w3,nn.w4,nn.w5,nn.w6,x_valid,y_valid)
        average_accuracy += accuracy
      average_accuracy = average_accuracy/5
      writer.writerow(["Score for", nodes, "nodes with a", rate, "learning rate", average_accuracy])

======Running a 5 layer Neural Network with 100  HIDDEN NODES and a  0.1  LEARNING RATE
ON CV  1
Currently on epoch:  0
(0.021, array([3, 3, 3, ..., 3, 3, 3]))
Currently on epoch:  10
(0.3855, array([24, 21, 21, ..., 15, 20, 13]))


KeyboardInterrupt: ignored

In [0]:
import csv
#write predictions to csv file
#(0,skull 1,pear ....)

with open("../predictions_NN.csv","w") as f:
  writer = csv.writer(f)
  id = 0
  writer.writerow(["Id","Category"])
  for line in yPredictions:
    result = [id,categories_dict[line]]
    writer.writerow(result)
    id += 1
    